### User input

In [ ]:
decimal = '.'                                                            # decimal of the input file
sep = ';'                                                                # separator of the input file

from config import *                                                     # Personal settings of local user to set input and output directories
input_dir = input_directory + 'results/df_shifted_sw.csv'                # input directory
output = output_directory + 'results/'                                   # output directory

dateheadername = 'Timestamp'                                             # header of the date  
header = 0                                                               # header row number
sample_name = 'sw'                                                       # name of the sample

path_length = 3.5                                                        # path length of the window of the sensor [cm]

### Start environment and import data

In [ ]:
import abspectroscopy_functions as abspy # Functions from the AbspectroscoPY toolbox
import tkinter as tk
from tkinter import messagebox
import pandas as pd

df = pd.read_csv(input_dir, sep = sep, header = header, index_col = 0) 
df.index = pd.to_datetime(df.index)      # make sure time column (here index) is using time format
df

### abs_pathcor

In [ ]:
def abs_pathcor(df_in, 
                path_length):
    '''
    function to correct the attenuation data according to the path length
    :argument df_in: dataframe in input
    :argument path_length: path length of the window of the sensor
    :return: the dataframe of attenuation data corrected by the path length of the window of the sensor    
    '''
    before_correction = df_in['255 nm'][0] # attenuation value read by the sensor for the first date before path length correction 
    abs_scale = path_length 
    nrcol = len(df_in.columns)
    df_out = df_in.copy()
    for i in range(0,nrcol):
        df_out.iloc[:,i] = df_out.iloc[:,i]/abs_scale
    pd.options.display.max_rows = 15
    after_correction = df_out['255 nm'][0] # attenuation value read by the sensor for the first date after path length correction 
    
    # Compare the attenuation value read by the sensor before and after path length correction for a specific date to the value 
    # obtained in the laboratory (consider that, if analysing filtered samples in the laboratory, part of the difference in the 
    # absolute value is due to the fact that the sensor measures instead unfiltered water: the sensor data should be higher than
    # the laboratory data):
    return('before correction:', before_correction, 'after correction:', after_correction, 'dataframe after correction:', df_out)

In [ ]:
abs_pathcor(df, path_length) [0:4]

In [ ]:
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

MsgBox = tk.messagebox.showwarning ('Warning','Compare the attenuation value read by the sensor before and after path length correction for a specific date to the value obtained in the laboratory. Input yes or no in the following cell.',icon = 'warning')
root.destroy()

In [ ]:
answer = input()
if answer == 'yes':
    df_pathcor = abs_pathcor(df, path_length)[5] # the dataframe with corrected values is the fifth output of the function
elif answer == 'no':
    df_pathcor = df.copy()
else:
    root = tk.Tk()
    root.withdraw()
    root.attributes("-topmost", True)

    MsgBox = tk.messagebox.showerror ('Error','You are not providing one of the two possible answers. Please input "yes" or "no".',icon = 'error')
    root.destroy()    

In [ ]:
df_pathcor.to_csv(output + 'df_pathcor_' + str(sample_name) + '.csv', sep = sep, decimal = decimal, index=True) # export the pathlength corrected dataset
df_pathcor